# Denna är inte samma som efterföjnade, kika på gtd200 osv

In [1]:
import numpy as np
import pandas as pd
import os
import networkx as nx
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from build_graph_data import *
from collections import Counter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [2]:
partition = 100

In [3]:
trainpath = f'../../../data/top30groups/LongLatCombined/train1/train{partition}.csv'
testpath = f'../../../data/top30groups/LongLatCombined/test1/test{partition}.csv'
traindata = pd.read_csv(trainpath, encoding='ISO-8859-1')
testdata = pd.read_csv(testpath, encoding='ISO-8859-1')

In [4]:
traindata.columns

Index(['extended', 'latitude', 'longitude', 'vicinity', 'multiple', 'success',
       'suicide', 'attacktype1', 'targtype1', 'target1', 'individual',
       'weaptype1', 'nkill', 'property', 'ishostkid', 'gname'],
      dtype='object')

In [5]:
combined = pd.concat([traindata, testdata], axis = 0)

In [6]:
combined.shape

(3000, 16)

In [7]:
data = combined

In [8]:
data.columns

Index(['extended', 'latitude', 'longitude', 'vicinity', 'multiple', 'success',
       'suicide', 'attacktype1', 'targtype1', 'target1', 'individual',
       'weaptype1', 'nkill', 'property', 'ishostkid', 'gname'],
      dtype='object')

In [9]:
# Filter dataset to only contain unique coordinates
print("Entries before dropping long/lat duplicates: ", data.shape)

#data.sort_values(by=['longitude', 'latitude', 'attack_date'], inplace=True)

# Keep only relevant columns
data = data[['longitude', 'latitude', 'gname']]
# Drop duplicates based on location, keep the earliest attack
data['longlat'] = list(zip(data['longitude'], data['latitude']))
df_unique = data.copy()

df_unique = df_unique.drop(columns=['longitude', 'latitude'])
print(df_unique.columns)
print("Entries after dropping long/lat duplicates (#Nodes): ", df_unique.shape)


Entries before dropping long/lat duplicates:  (3000, 16)
Index(['gname', 'longlat'], dtype='object')
Entries after dropping long/lat duplicates (#Nodes):  (3000, 2)


/tmp/ipykernel_17633/345096426.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['longlat'] = list(zip(data['longitude'], data['latitude']))


In [10]:
df_unique["gname"].value_counts()

gname
Shining Path (SL)                                   100
Abu Sayyaf Group (ASG)                              100
Liberation Tigers of Tamil Eelam (LTTE)             100
Revolutionary Armed Forces of Colombia (FARC)       100
Communist Party of India - Maoist (CPI-Maoist)      100
National Liberation Army of Colombia (ELN)          100
African National Congress (South Africa)            100
Nicaraguan Democratic Force (FDN)                   100
Basque Fatherland and Freedom (ETA)                 100
New People's Army (NPA)                             100
Palestinians                                        100
Houthi extremists (Ansar Allah)                     100
Kurdistan Workers' Party (PKK)                      100
Fulani extremists                                   100
Al-Qaida in Iraq                                    100
Al-Qaida in the Arabian Peninsula (AQAP)            100
Sikh Extremists                                     100
Manuel Rodriguez Patriotic Front (FPMR)   

In [11]:
split_point = int(0.7 * len(df_unique))
train_df = df_unique[:split_point]
test_df = df_unique[split_point:]

In [12]:
le = LabelEncoder()
df_unique['label'] = le.fit_transform(df_unique['gname'])


In [13]:
df_unique['longitude'], df_unique['latitude'] = zip(*df_unique['longlat'])
coords = df_unique[['longitude', 'latitude']].values

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features = scaler.fit_transform(coords)  # shape: [num_nodes, 2]


In [14]:
from haversine import haversine, Unit

# Make sure you have access to lon/lat from df_unique
coords = list(zip(df_unique['latitude'], df_unique['longitude']))  # Haversine uses (lat, lon)

distance_threshold_km = 1  # You can change this to your needs
edges = []

num_nodes = len(coords)
for i in range(num_nodes):
    for j in range(i + 1, num_nodes):
        dist = haversine(coords[i], coords[j], unit=Unit.KILOMETERS)
        if dist <= distance_threshold_km:
            edges.append((i, j))
            edges.append((j, i))  # undirected graph


In [15]:
num_nodes = len(df_unique)
y = df_unique['label'].values
train_mask = np.zeros(num_nodes, dtype=bool)
test_mask = np.zeros(num_nodes, dtype=bool)

train_indices = train_df.index
test_indices = test_df.index

train_mask[train_indices] = True
test_mask[test_indices] = True


In [16]:
import torch
from torch_geometric.data import Data

x = torch.tensor(features, dtype=torch.float)
y = torch.tensor(y, dtype=torch.long)
edge_index = torch.tensor(np.array(edges).T, dtype=torch.long)
train_mask = torch.tensor(train_mask)
test_mask = torch.tensor(test_mask)

dataobj = Data(x=x, edge_index=edge_index, y=y)
dataobj.train_mask = train_mask
dataobj.test_mask = test_mask


/opt/conda/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [17]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, dataobj):
        x, edge_index = dataobj.x, dataobj.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [18]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GCN(
    in_channels=dataobj.num_node_features,
    hidden_channels=64,
    out_channels=int(y.max().item()) + 1  # number of classes
).to(device)

dataobj = dataobj.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


In [19]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(dataobj)
    loss = criterion(out[dataobj.train_mask], dataobj.y[dataobj.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def test(mask):
    model.eval()
    out = model(dataobj)
    pred = out[mask].argmax(dim=1)
    correct = (pred == dataobj.y[mask]).sum().item()
    return correct / mask.sum().item()


In [20]:
for epoch in range(1, 201):
    loss = train()
    if epoch % 10 == 0:
        train_acc = test(dataobj.train_mask)
        test_acc = test(dataobj.test_mask)
        print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")


Epoch 010, Loss: 2.7411, Train Acc: 0.1976, Test Acc: 0.2000
Epoch 020, Loss: 2.1671, Train Acc: 0.3000, Test Acc: 0.3056
Epoch 030, Loss: 1.7768, Train Acc: 0.5190, Test Acc: 0.5011
Epoch 040, Loss: 1.4996, Train Acc: 0.6476, Test Acc: 0.6367
Epoch 050, Loss: 1.2907, Train Acc: 0.6652, Test Acc: 0.6567
Epoch 060, Loss: 1.1374, Train Acc: 0.7367, Test Acc: 0.7233
Epoch 070, Loss: 1.0232, Train Acc: 0.7533, Test Acc: 0.7500
Epoch 080, Loss: 0.9367, Train Acc: 0.7576, Test Acc: 0.7500
Epoch 090, Loss: 0.8706, Train Acc: 0.7567, Test Acc: 0.7511
Epoch 100, Loss: 0.8191, Train Acc: 0.7633, Test Acc: 0.7600
Epoch 110, Loss: 0.7781, Train Acc: 0.7657, Test Acc: 0.7611
Epoch 120, Loss: 0.7446, Train Acc: 0.7700, Test Acc: 0.7644
Epoch 130, Loss: 0.7167, Train Acc: 0.7795, Test Acc: 0.7767
Epoch 140, Loss: 0.6927, Train Acc: 0.7852, Test Acc: 0.7800
Epoch 150, Loss: 0.6714, Train Acc: 0.7890, Test Acc: 0.7844
Epoch 160, Loss: 0.6518, Train Acc: 0.7919, Test Acc: 0.7889
Epoch 170, Loss: 0.6342,

In [21]:
# Classification report for test set
model.eval()
with torch.no_grad():
    out = model(dataobj)
    pred = out[dataobj.test_mask].argmax(dim=1)
    y_true = dataobj.y[dataobj.test_mask].cpu().numpy()
    y_pred = pred.cpu().numpy()

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=le.classes_))


                                                  precision    recall  f1-score   support

                          Abu Sayyaf Group (ASG)       0.77      0.97      0.86        31
        African National Congress (South Africa)       1.00      0.97      0.98        31
                                Al-Qaida in Iraq       0.65      0.80      0.72        30
        Al-Qaida in the Arabian Peninsula (AQAP)       0.72      0.79      0.75        29
                                      Al-Shabaab       1.00      1.00      1.00        28
             Basque Fatherland and Freedom (ETA)       1.00      0.94      0.97        32
                                      Boko Haram       0.91      0.95      0.93        22
  Communist Party of India - Maoist (CPI-Maoist)       0.68      0.76      0.72        25
       Corsican National Liberation Front (FLNC)       0.87      1.00      0.93        33
                       Donetsk People's Republic       0.91      1.00      0.95        31
Farabundo